# Problem Set #3 (Task #1)
## Name: Sahithi Adari
### Date: 10/19/2020

In [2]:
import pandas as pd
import numpy as np
import country_converter as coco
import requests # For downloading the website
from bs4 import BeautifulSoup # For parsing the website
import time # To put the system to sleep
import random # for random numbers
import sqlite3

## Task 1: Data Acquisition

### Country Level Dataset

In [3]:
#Scraped the table of bilateral investment treaties and saved this as a 'main_url' variable
main_url = 'https://investmentpolicy.unctad.org/international-investment-agreements/by-economy'

#Made a get request for the 'main_url'
main_page = requests.get(main_url)

#Read the HTML tables into a list of DataFrame objects
bit = pd.read_html(main_page.text)

#Resaved the 0th index of 'bit' as dataframe to get the complete BIT table
bit_table = bit[0]

#Removed the 'No.' and 'Total TIPs' columns from the data
bit_table = bit_table.drop(columns = ['No.', '*  TOTAL TIPs'])

#Renamed the remaining 2 columns
bit_table = bit_table.rename(columns={"Name": "temp_country", "*  TOTAL BITs": "n_bits"})

In [4]:
#Created a new dataframe that would split 'n_bits' into 2 values
paren = bit_table["n_bits"].str.split("(", n = 1, expand = True)

#Resaved 'n_bits' as the first column of 'paren'
bit_table["n_bits"] = paren[0]

#Saved the second column as 'n_bits_active'
bit_table["n_bits_active"] = paren[1]

#Created a newdataframe that would split 'n_bits_active' into 2 values
space = bit_table["n_bits_active"].str.split(" ", n = 1, expand = True)

#Reasaved 'n_bit_active' as the first column of 'space' and filled NA values as 0
bit_table["n_bits_active"] = space[0].fillna(0)

In [5]:
#Change the datatype for all 3 columns of "bit_table"
bit_table['temp_country'] = bit_table['temp_country'].astype('string')
bit_table['n_bits'] = bit_table['n_bits'].astype('int')
bit_table['n_bits_active'] = bit_table['n_bits_active'].astype('int')

In [6]:
#Created a new list from the first column
bit_country = bit_table.temp_country.tolist()

#Country-converter object to restrict the set to only the official recognized UN members
cc_UN = coco.CountryConverter(only_UNmember=True)

#Ran cc_UN on 'bit_country' and saved that list as 'UN'
UN = cc_UN.convert(bit_country, to = 'name_short')

#Dropped the 'temp_country' column from the dataframe
bit_table = bit_table.drop(columns = "temp_country")

Anguilla not found in regex
Aruba not found in regex
Bermuda not found in regex
British Virgin Islands not found in regex
Cayman Islands not found in regex
Channel Islands not found in regex
Christmas Island not found in regex
Cocos (Keeling) Islands not found in regex
Cook Islands not found in regex
Curaçao not found in regex
Faeroe Islands not found in regex
Falkland Islands (Malvinas) not found in regex
French Guiana not found in regex
French Polynesia not found in regex
Gibraltar not found in regex
Greenland not found in regex
Guadeloupe not found in regex
Guam not found in regex
Holy See not found in regex
Hong Kong, China SAR not found in regex
Isle of Man not found in regex
Macao, China SAR not found in regex
Martinique not found in regex
Mayotte not found in regex
Montserrat not found in regex
New Caledonia not found in regex
Niue not found in regex
Norfolk Island not found in regex
Northern Mariana Islands not found in regex
Pitcairn not found in regex
Puerto Rico not found in

Because 'cc_UN' replaces any country that is not a UN member nation with "not found" I felt fairly comfortable in dropping the original *temp_country* column from *bit_table*. I replaced this column with values from the *UN* list and called this new column *country* as the order will still be the same.

In [7]:
#Created a new column called 'country' and set those values as 'UN'
bit_table["country"] = UN

#Set the index as the 'country' column
bit_table = bit_table.set_index('country')

#Dropped any rows that had a "not found" value
bit_table = bit_table.drop(index = "not found")

#Reset the index in order to remove the "country" column as the index
bit_table = bit_table.reset_index()

### Dyad Level Dataset 

In [8]:
#Parsed the data using BeautifulSoup
main_soup = BeautifulSoup(main_page.content,'html.parser')

#Extracted relevant links using a for loop
links = [] #Created an empty list to hold our relevent links
for tag in main_soup.find_all("a"): #Used BeautifulSoup to find all the <a> tags in the HTML code
    href = tag.attrs.get("href") #Saved the 'href' tag of all links as 'href'
    if "countries" in href:
        links.append("https://investmentpolicy.unctad.org" + href) #Appended unique 'href' values to the 'links' list

In [9]:
#Created a funtion to pull all relevent links from the webite and to capture the tables on the wesite only
def unctad_scraper(url = None):
    '''
    Scraped single links on the UNCTAD site.

    Args:
        urls (list): list of valid UNCTAD urls to be processed one at a time.

    Returns:
        DataFrame: frame containing scraped tables
    '''
    page = requests.get(url) #Downloaded the webpage
    
    unctad_table = None #Created an empty variable to capture our tables
    
    #If a connection was reached
    if page.status_code == 200:
        #Parse by reading the HTML tables into a list of DataFrame objects
        try:
            unctad_country = pd.read_html(page.text)
            unctad_table = unctad_country[0] #Resaved the 0th index of 'unctad_country' as list to get the complete table
        #If the link has no table, print the link with the no data and continue
        except Exception as e:
            print('No data from URL:', url)
    return unctad_table

#Created an empty list to capture all the tables from the 'unctad_scraper' function
scraped_data = []

#For the urls found in the 'links' list
for url in links:
    print(url)
    scraped_data.append(unctad_scraper(url)) #Scraped the content by passing the through the 'unctad_scraper' function and appended to the new list
    time.sleep(random.uniform(1,5)) #Put the system to sleep for a random draw of time between 1-5 seconds

https://investmentpolicy.unctad.org/international-investment-agreements/countries/1/afghanistan
https://investmentpolicy.unctad.org/international-investment-agreements/countries/2/albania
https://investmentpolicy.unctad.org/international-investment-agreements/countries/3/algeria
https://investmentpolicy.unctad.org/international-investment-agreements/countries/4/andorra
https://investmentpolicy.unctad.org/international-investment-agreements/countries/5/angola
https://investmentpolicy.unctad.org/international-investment-agreements/countries/6/anguilla
https://investmentpolicy.unctad.org/international-investment-agreements/countries/7/antigua-and-barbuda
https://investmentpolicy.unctad.org/international-investment-agreements/countries/8/argentina
https://investmentpolicy.unctad.org/international-investment-agreements/countries/9/armenia
https://investmentpolicy.unctad.org/international-investment-agreements/countries/10/aruba
https://investmentpolicy.unctad.org/international-investment-ag

https://investmentpolicy.unctad.org/international-investment-agreements/countries/80/gibraltar
No data from URL: https://investmentpolicy.unctad.org/international-investment-agreements/countries/80/gibraltar
https://investmentpolicy.unctad.org/international-investment-agreements/countries/81/greece
https://investmentpolicy.unctad.org/international-investment-agreements/countries/82/greenland
https://investmentpolicy.unctad.org/international-investment-agreements/countries/83/grenada
https://investmentpolicy.unctad.org/international-investment-agreements/countries/84/guadeloupe
No data from URL: https://investmentpolicy.unctad.org/international-investment-agreements/countries/84/guadeloupe
https://investmentpolicy.unctad.org/international-investment-agreements/countries/85/guam
No data from URL: https://investmentpolicy.unctad.org/international-investment-agreements/countries/85/guam
https://investmentpolicy.unctad.org/international-investment-agreements/countries/86/guatemala
https://i

https://investmentpolicy.unctad.org/international-investment-agreements/countries/156/northern-mariana-islands
No data from URL: https://investmentpolicy.unctad.org/international-investment-agreements/countries/156/northern-mariana-islands
https://investmentpolicy.unctad.org/international-investment-agreements/countries/157/norway
https://investmentpolicy.unctad.org/international-investment-agreements/countries/159/oman
https://investmentpolicy.unctad.org/international-investment-agreements/countries/160/pakistan
https://investmentpolicy.unctad.org/international-investment-agreements/countries/161/palau
https://investmentpolicy.unctad.org/international-investment-agreements/countries/162/panama
https://investmentpolicy.unctad.org/international-investment-agreements/countries/163/papua-new-guinea
https://investmentpolicy.unctad.org/international-investment-agreements/countries/164/paraguay
https://investmentpolicy.unctad.org/international-investment-agreements/countries/165/peru
https:/

https://investmentpolicy.unctad.org/international-investment-agreements/countries/233/zimbabwe


In [10]:
#Saved the 0th index of the scraped data to get there first dataframe in 'scraped_data' as 'full_bit'
full_bit = scraped_data[0]

#For a range of 1 to the total length of the 'scraped_data' list
for r in range(1, len(scraped_data)):
    #Appended the rth index to the 'full_bit' in order to get all pertinent dataframes
    full_bit = full_bit.append(scraped_data[r], ignore_index= True)

In [11]:
#Removed rows with NA in 'Type' column and resaved the dataframe as 'full_bit'
full_bit = full_bit[full_bit.Type.notna()]

#Set the index as the 'Type' column
full_bit = full_bit.set_index('Type')

#Dropped any rows that had a "TIPs", "Intergovernmental agreements", "Draft instruments", and "Guidelines, principles, resolutions and similar" values
full_bit = full_bit.drop(index = ["TIPs",
                                  "Guidelines, principles, resolutions and similar",
                                  "Intergovernmental agreements",
                                  "Draft instruments"])

#Reset the index in order to remove the "Type" column as the index
full_bit = full_bit.reset_index()

#Removed the 0, 'Full title', 'No.', 'Text', 'Level', 'Files', 'Date of adoption', 'Type' columns from the data
full_bit = full_bit.drop(columns = ["Full title", "No.", "Text", "Level", "Files", "Date of adoption", "Type"])

#Renamed the remaining 6 columns
full_bit = full_bit.rename(columns={"Short title": "temp_country",
                                    "Parties": "temp_country_B",
                                    "Status": "status",
                                    "Date of signature": "year_signed",
                                    "Date of entry into force" : "year_enforced",
                                    "Termination date" : "year_terminated"})

In [12]:
#Created a new dataframe that would split 'temp_country' into 2 values
dash = full_bit["temp_country"].str.split("-", n = 1, expand = True)

#Saved 'country_A1' as the first column of 'dash' and removed any spaces from the beginning and end
full_bit["temp_country_A1"] = dash[0].str.strip()

#Saved 'country_A2' as the second column of 'dash', split this new value by BIT, took the 0th value of this new split, and removed spaces
full_bit["temp_country_A2"] = dash[1].str.split("BIT", n = 1, expand = True)[0].str.strip()

#Removed the 'temp_country' columns from the data
full_bit = full_bit.drop(columns = ["temp_country"])

I created a for loop to discern if the values of *temp_country_A1* or *temp_country_A2* equals *temp_country_B*. If an equality between one column and *temp_country_B*, the for loop with take the value in the other column and save that in *temp_country_A*.

In [13]:
#Created an empty column for the dataframe
full_bit['temp_country_A'] = ''

#For a range of the total length of 'full_bit'
for a in range(len(full_bit)): #Set the range as the length of the dataframe
    #Checked to see if the values of 'temp_country_A1' equals 'temp_country_B'
    if full_bit.temp_country_A1[a] == full_bit.temp_country_B[a]:
        #If the above condition holds set value for the column of 'temp_country_A' as 'temp_country_A2'
        full_bit['temp_country_A'][a] = full_bit.temp_country_A2[a]
    else:
        full_bit['temp_country_A'][a] = full_bit.temp_country_A1[a]

#Removed the 'temp_country_A1' and 'temp_country_A2' columns from the data
full_bit = full_bit.drop(columns = ['temp_country_A1', 'temp_country_A2']) 

In [14]:
#Created a new list from the 'temp_country_A' column
bit_country_A = full_bit.temp_country_A.tolist()

#Ran cc_UN on 'bit_country_A' and saved that list as 'UN_A'
UN_A = cc_UN.convert(bit_country_A, to = 'name_short')

#Dropped the 'temp_country_A' column from the dataframe
full_bit = full_bit.drop(columns = "temp_country_A")

#Created a new column called 'country_A' and set those values as 'UN_A'
full_bit["country_A"] = UN_A

#Set the index as the 'country_A' column
full_bit = full_bit.set_index('country_A')

#Dropped any rows that had a "not found" value
full_bit = full_bit.drop(index = "not found")

#Reset the index in order to remove the "country_A" column as the index
full_bit = full_bit.reset_index()

BLEU (Belgium-Luxembourg Economic Union) not found in regex
BLEU (Belgium-Luxembourg Economic Union) not found in regex
BLEU (Belgium-Luxembourg Economic Union) not found in regex
BLEU (Belgium-Luxembourg Economic Union) not found in regex
BLEU (Belgium-Luxembourg Economic Union) not found in regex
BLEU (Belgium-Luxembourg Economic Union) not found in regex
BLEU (Belgium-Luxembourg Economic Union) not found in regex
BLEU (Belgium-Luxembourg Economic Union) not found in regex
Belgium/Luxembourg not found in regex
Belgium/Luxembourg not found in regex
Belgium/Luxembourg not found in regex
Belgium/Luxembourg not found in regex
Belgium/Luxembourg not found in regex
BLEU (Belgium-Luxembourg Economic Union) not found in regex
Hong Kong, China SAR not found in regex
Hong Kong, China SAR not found in regex
Hong Kong, China SAR not found in regex
Hong Kong, China SAR not found in regex
Hong Kong, China SAR not found in regex
Hong Kong, China SAR not found in regex
Hong Kong, China SAR not found

In [15]:
#Created a new list from the 'temp_country_B' column
bit_country_B = full_bit.temp_country_B.tolist()

#Ran cc_UN on 'bit_country_B' and saved that list as 'UN_B'
UN_B = cc_UN.convert(bit_country_B, to = 'name_short')

#Dropped the 'temp_country_B' column from the dataframe
full_bit = full_bit.drop(columns = "temp_country_B")

#Created a new column called 'country_B' and set those values as 'UN_B'
full_bit["country_B"] = UN_B

#Set the index as the 'country_B' column
full_bit = full_bit.set_index('country_B')

#Dropped any rows that had a "not found" value
full_bit = full_bit.drop(index = "not found")

#Reset the index in order to remove the "country_B" column as the index
full_bit = full_bit.reset_index()

BLEU (Belgium-Luxembourg Economic Union) not found in regex
BLEU (Belgium-Luxembourg Economic Union) not found in regex
BLEU (Belgium-Luxembourg Economic Union) not found in regex
BLEU (Belgium-Luxembourg Economic Union) not found in regex
Hong Kong, China SAR not found in regex
Hong Kong, China SAR not found in regex
BLEU (Belgium-Luxembourg Economic Union) not found in regex
BLEU (Belgium-Luxembourg Economic Union) not found in regex
BLEU (Belgium-Luxembourg Economic Union) not found in regex
BLEU (Belgium-Luxembourg Economic Union) not found in regex
BLEU (Belgium-Luxembourg Economic Union) not found in regex
Hong Kong, China SAR not found in regex
Taiwan Province of China not found in regex
BLEU (Belgium-Luxembourg Economic Union) not found in regex
BLEU (Belgium-Luxembourg Economic Union) not found in regex
BLEU (Belgium-Luxembourg Economic Union) not found in regex
BLEU (Belgium-Luxembourg Economic Union) not found in regex
BLEU (Belgium-Luxembourg Economic Union) not found in re

BLEU (Belgium-Luxembourg Economic Union) not found in regex


In [16]:
#Replaced the values of the 'Status' columns
full_bit["status"] = full_bit["status"].replace({"Signed (not in force)": "signed",
                            'In force': 'active',
                            'Terminated': 'terminated'})

#Changed the years column to a DateTimeIndex and got the year
full_bit['year_enforced'] = pd.DatetimeIndex(full_bit['year_enforced']).year
full_bit['year_signed'] = pd.DatetimeIndex(full_bit['year_signed']).year
full_bit['year_terminated'] = pd.DatetimeIndex(full_bit['year_terminated']).year

#Changed the datatype for all 6 columns of "full_bit"
full_bit["country_A"] = full_bit["country_A"].astype('string')
full_bit["country_B"] = full_bit["country_B"].astype('string')
full_bit["status"] = full_bit["status"].astype('string')
full_bit['year_enforced'] = full_bit['year_enforced'].astype('Int64')
full_bit['year_signed'] = full_bit['year_signed'].astype('Int64')
full_bit['year_terminated'] = full_bit['year_terminated'].astype('Int64')

#Rearranged the column orders for "full_bit"
full_bit = full_bit[['country_A', 'country_B', 'status', 'year_signed', 'year_enforced', 'year_terminated']]

In [17]:
#Created a SQL connection to our SQLite database
con = sqlite3.connect("ps3_db.sqlite")

#Wrote the 'bit_table' DataFrame to a new SQLite table with 'ps3_db_scratch'
bit_table.to_sql("country_level", con, if_exists="replace")

#Wrote the 'full_bit' DataFrame to a new SQLite table with 'ps3_db_scratch'
full_bit.to_sql("dyad_level", con, if_exists="replace")

#Closed the connection
con.close()